In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import warnings 
warnings.filterwarnings("ignore")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!git clone -b feature/add_hf https://github.com/nielsrogge/ImageBind.git
%cd ImageBind

Cloning into 'ImageBind'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 125 (delta 57), reused 39 (delta 39), pack-reused 40 (from 1)
Receiving objects: 100% (125/125), 2.64 MiB | 31.79 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/kaggle/working/ImageBind


In [3]:
!pip install .

Processing /kaggle/working/ImageBind
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/facebookresearch/pytorchvideo.git (to revision 28fe037d212663c6a24f373b94cc5d478c8c1a1d) to /tmp/pip-install-wqu_7uj_/pytorchvideo_d1f82f7670304233bb38dd06b1ab51d0
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/pytorchvideo.git /tmp/pip-install-wqu_7uj_/pytorchvideo_d1f82f7670304233bb38dd06b1ab51d0
  Running command git rev-parse -q --verify 'sha^28fe037d212663c6a24f373b94cc5d478c8c1a1d'
  Running command git fetch -q https://github.com/facebookresearch/pytorchvideo.git 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Running command git checkout -q 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Resolved https://github.com/facebookresearch/pytorchvideo.git to commit 28fe037d212663c6a24f373b94cc5d478c8c1a1d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (

In [4]:
from imagebind import data
import torch
from imagebind.models import imagebind_model
from imagebind.models.imagebind_model import ModalityType
from imagebind.models.imagebind_model import ImageBindModel

text_list=["A dog.", "A car", "A bird"]
image_paths=[".assets/dog_image.jpg", ".assets/car_image.jpg", ".assets/bird_image.jpg"]
audio_paths=[".assets/dog_audio.wav", ".assets/car_audio.wav", ".assets/bird_audio.wav"]

device = "cuda:0" if torch.cuda.is_available() else "cpu"

model = ImageBindModel.from_pretrained("nielsr/imagebind-huge")
model.eval()
model.to(device)


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.80G [00:00<?, ?B/s]

ImageBindModel(
  (modality_preprocessors): ModuleDict(
    (vision): RGBDTPreprocessor(
      (cls_token): tensor((1, 1, 1280), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Sequential(
          (0): PadIm2Video()
          (1): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
        )
      )
      (pos_embedding_helper): SpatioTemporalPosEmbeddingHelper(
        (pos_embed): tensor((1, 257, 1280), requires_grad=True)
        
      )
    )
    (text): TextPreprocessor(
      (pos_embed): tensor((1, 77, 1024), requires_grad=True)
      (mask): tensor((77, 77), requires_grad=False)
      
      (token_embedding): Embedding(49408, 1024)
    )
    (audio): AudioPreprocessor(
      (cls_token): tensor((1, 1, 768), requires_grad=True)
      
      (rgbt_stem): PatchEmbedGeneric(
        (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10), bias=False)
        (norm_layer): LayerNorm((768,), eps=1e-05, elementwise_affine=

# Full Knowledge

In [5]:
import ast
input_df = pd.read_csv('/kaggle/input/business-json/Task_2_data.csv') 
# Function to extract entity names
def extract_entity_name(entities_str):
    entities_list = ast.literal_eval(entities_str)
    # Return the first two mentions if they exist, otherwise None
    return (entities_list[0], entities_list[1]) if len(entities_list) > 1 else (entities_list[0], None)

# Apply the function to the 'entities' column and expand into two new columns
input_df[['sentence_1_entity_name', 'sentence_2_entity_name']] = input_df['entity_titles'].apply(extract_entity_name).apply(pd.Series)

# Display the updated DataFrame
print(input_df)
entity_df = pd.read_csv('/kaggle/input/business-json/Knowledge.csv')
asr_df = pd.read_csv("/kaggle/input/business-json/task_2_asr_out.csv")

     Unnamed: 0  QID                                   question  \
0             0    0  Are these established in the same decade?   
1             1    1  Are these established in the same decade?   
2             2    2  Are these established in the same decade?   
3             3    3  Are these established in the same decade?   
4             4    4  Are these established in the same decade?   
..          ...  ...                                        ...   
495         495  495               Are these greek restaurants?   
496         496  496               Are these greek restaurants?   
497         497  497               Are these greek restaurants?   
498         498  498               Are these greek restaurants?   
499         499  499               Are these greek restaurants?   

                                          sentence_1  \
0      Banco do Brasil has headquarters in Brasília.   
1         Banco do Brasil has origin country Brazil.   
2       Banco do Brasil has

In [6]:
import os
import torch
from imagebind.models.imagebind_model import ModalityType, ImageBindModel
from imagebind import data
import pandas as pd
from tqdm import tqdm
device = "cuda:0" if torch.cuda.is_available() else "cpu"
from sklearn.metrics.pairwise import cosine_similarity

# Function to process audio files in batches
def process_audio_in_batches(txt, batch_size=16):
    all_embeddings = []
    for i in tqdm(range(0, len(txt), batch_size), desc="Processing Batches"):
        batch_txt = txt[i:i + batch_size]
        embed_inputs = data.load_and_transform_text(batch_txt, device)
        
        with torch.no_grad():
            embeddings = model({
                ModalityType.TEXT: embed_inputs,
            })
        
        all_embeddings.append(embeddings[ModalityType.TEXT])
        torch.cuda.empty_cache()
    
    return torch.cat(all_embeddings, dim = 0)

# # Process entity audio files in batches
entity_embeddings= process_audio_in_batches(entity_df['Knowledge'].tolist())

# Process sentence audio files in batches
input1_embeddings  = process_audio_in_batches(asr_df['Sentence_1_transcript'].tolist())
input2_embeddings = process_audio_in_batches(asr_df['Sentence_2_transcript'].tolist())


Processing Batches: 100%|██████████| 32/32 [00:08<00:00,  3.59it/s]


In [7]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

top1_correct_s = 0
top5_correct_s = 0
top10_correct_s = 0

r_s = []
p_s = []
f1_s = []
res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    count = 0
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1
    if actual_entity_name1 == ranked_entity_names1.iloc[0] and actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values and actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values and actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct_s += 1
    
    r = p = f1 = count/2
    
    r_s.append(r)
    p_s.append(p)
    f1_s.append(f1)
# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50


top1_accuracy_s = top1_correct_s / total_sentences * 100
top5_accuracy_s = top5_correct_s / total_sentences * 100
top10_accuracy_s = top10_correct_s / total_sentences * 100


print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")


print(f"Top-1 Set Accuracy: {top1_accuracy_s:.2f}%")
print(f"Top-5 Set Accuracy: {top5_accuracy_s:.2f}%")
print(f"Top-10 Set Accuracy: {top10_accuracy_s:.2f}%")


print(f'average f1 score : {(sum(f1_s)/len(f1_s))}')
print(f'average recall score : {(sum(r_s)/len(r_s))}')
print(f'average precision score : {(sum(p_s)/len(p_s))}')

Top-1 Accuracy: 62.70%
Top-5 Accuracy: 75.10%
Top-10 Accuracy: 78.20%
Top-1 Set Accuracy: 38.60%
Top-5 Set Accuracy: 56.80%
Top-10 Set Accuracy: 61.80%
average f1 score : 0.627
average recall score : 0.627
average precision score : 0.627


In [8]:
ner_df = pd.DataFrame(res_data)
ner_df.to_csv("Task2_asr_entity_linked_ImageBIND_results.csv")

# Entity Label

In [9]:

# # Process entity audio files in batches
entity_embeddings = process_audio_in_batches(entity_df['Entity_name'].tolist())

# Process sentence audio files in batches
input1_embeddings  = process_audio_in_batches(asr_df['Sentence_1_transcript'].tolist())
input2_embeddings = process_audio_in_batches(asr_df['Sentence_2_transcript'].tolist())



Processing Batches: 100%|██████████| 32/32 [00:09<00:00,  3.50it/s]


In [10]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

top1_correct_s = 0
top5_correct_s = 0
top10_correct_s = 0

r_s = []
p_s = []
f1_s = []
res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    count = 0
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1
    if actual_entity_name1 == ranked_entity_names1.iloc[0] and actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values and actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values and actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct_s += 1
    
    r = p = f1 = count/2
    
    r_s.append(r)
    p_s.append(p)
    f1_s.append(f1)
# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50


top1_accuracy_s = top1_correct_s / total_sentences * 100
top5_accuracy_s = top5_correct_s / total_sentences * 100
top10_accuracy_s = top10_correct_s / total_sentences * 100


print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")


print(f"Top-1 Set Accuracy: {top1_accuracy_s:.2f}%")
print(f"Top-5 Set Accuracy: {top5_accuracy_s:.2f}%")
print(f"Top-10 Set Accuracy: {top10_accuracy_s:.2f}%")


print(f'average f1 score : {(sum(f1_s)/len(f1_s))}')
print(f'average recall score : {(sum(r_s)/len(r_s))}')
print(f'average precision score : {(sum(p_s)/len(p_s))}')

Top-1 Accuracy: 65.60%
Top-5 Accuracy: 79.90%
Top-10 Accuracy: 83.80%
Top-1 Set Accuracy: 42.80%
Top-5 Set Accuracy: 63.60%
Top-10 Set Accuracy: 69.60%
average f1 score : 0.656
average recall score : 0.656
average precision score : 0.656


In [11]:
ner_df = pd.DataFrame(res_data)
ner_df.to_csv("Task2_asr_entity_linked_ImageBIND_label_results.csv")

# Partial Knowledge

In [12]:
# Calculate the substring for the first 20% of each knowledge string
entity_df['Knowledge_20'] = entity_df['Knowledge'].apply(lambda x: x[:int(len(x) * 0.2)])

# # Process entity audio files in batches
entity_embeddings = process_audio_in_batches(entity_df['Knowledge_20'].tolist())

# Process sentence audio files in batches
input1_embeddings  = process_audio_in_batches(asr_df['Sentence_1_transcript'].tolist())
input2_embeddings = process_audio_in_batches(asr_df['Sentence_2_transcript'].tolist())



Processing Batches: 100%|██████████| 32/32 [00:09<00:00,  3.42it/s]


In [13]:
# Initialize counters for ranking accuracy
top1_correct = 0
top5_correct = 0
top10_correct = 0

top1_correct_s = 0
top5_correct_s = 0
top10_correct_s = 0

r_s = []
p_s = []
f1_s = []
res_data = []

# Step 3: Calculate cosine similarity and evaluate rankings
for i, q_id in enumerate(input_df['QID']):
    # Get the similarity scores for the current sentence with all entities
    similarity_scores1 = cosine_similarity(input1_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    similarity_scores2 = cosine_similarity(input2_embeddings[i].unsqueeze(0).cpu().numpy(), entity_embeddings.cpu().numpy()).flatten()
    # Get the indices of entities sorted by similarity score (descending order)
    ranked_entity_indices1 = similarity_scores1.argsort()[::-1]
    ranked_entity_indices2 = similarity_scores2.argsort()[::-1]
    # Get the ranked entity names
    ranked_entity_names1 = entity_df['Entity_name'].iloc[ranked_entity_indices1]
    ranked_entity_names2 = entity_df['Entity_name'].iloc[ranked_entity_indices2]

    # Get the actual entity name for the current sentence
    actual_entity_name1 = input_df['sentence_1_entity_name'].iloc[i]
    actual_entity_name2 = input_df['sentence_2_entity_name'].iloc[i]
    
    res_data.append(
    {
        'q_id': q_id,
        'linked_entity_sentnece_1': ranked_entity_names1.iloc[0],
        'actual_entity_sentence_1' : actual_entity_name1,
        'linked_entity_sentnece_2': ranked_entity_names2.iloc[0],
        'actual_entity_sentence_2' : actual_entity_name2,
    }
    )
    count = 0
    # Step 4: Check if the actual entity is within the top 1, 5, and 10
    if actual_entity_name1 == ranked_entity_names1.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values:
        top10_correct += 1
        
    if actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct += 1
        count += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct += 1
    if actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct += 1
    if actual_entity_name1 == ranked_entity_names1.iloc[0] and actual_entity_name2 == ranked_entity_names2.iloc[0]:
        top1_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:5].values and actual_entity_name2 in ranked_entity_names2.iloc[:5].values:
        top5_correct_s += 1
    if actual_entity_name1 in ranked_entity_names1.iloc[:10].values and actual_entity_name2 in ranked_entity_names2.iloc[:10].values:
        top10_correct_s += 1
    
    r = p = f1 = count/2
    
    r_s.append(r)
    p_s.append(p)
    f1_s.append(f1)
# Step 5: Calculate and print ranking accuracies
total_sentences = len(input_df)

top1_accuracy = top1_correct / total_sentences * 50
top5_accuracy = top5_correct / total_sentences * 50
top10_accuracy = top10_correct / total_sentences * 50


top1_accuracy_s = top1_correct_s / total_sentences * 100
top5_accuracy_s = top5_correct_s / total_sentences * 100
top10_accuracy_s = top10_correct_s / total_sentences * 100


print(f"Top-1 Accuracy: {top1_accuracy:.2f}%")
print(f"Top-5 Accuracy: {top5_accuracy:.2f}%")
print(f"Top-10 Accuracy: {top10_accuracy:.2f}%")


print(f"Top-1 Set Accuracy: {top1_accuracy_s:.2f}%")
print(f"Top-5 Set Accuracy: {top5_accuracy_s:.2f}%")
print(f"Top-10 Set Accuracy: {top10_accuracy_s:.2f}%")


print(f'average f1 score : {(sum(f1_s)/len(f1_s))}')
print(f'average recall score : {(sum(r_s)/len(r_s))}')
print(f'average precision score : {(sum(p_s)/len(p_s))}')

Top-1 Accuracy: 62.60%
Top-5 Accuracy: 78.20%
Top-10 Accuracy: 82.80%
Top-1 Set Accuracy: 38.60%
Top-5 Set Accuracy: 62.20%
Top-10 Set Accuracy: 69.20%
average f1 score : 0.626
average recall score : 0.626
average precision score : 0.626


In [14]:
ner_df = pd.DataFrame(res_data)
ner_df.to_csv("Task2_asr_entity_linked_ImageBIND_partial_knowledge_results.csv")